# Adding a Custom Pipeline Component on Spacy: Normalizing Token Spelling 

In handwritten documents, and particularly those in early modern Spanish, there is a wide range of spelling variation that will affect the representation of each token, and thus the performance of our model. Here, we provide a way to incorporate a dictionary that standardizes spelling through tokens' NORM attribute (as recommended by [Spacy creators](https://stackoverflow.com/questions/49493232/how-to-add-custom-slangs-into-spacys-norm-exceptions-py-module)).

In this document, we modify Spacy's pipeline, creating a custom component that normalizes word spelling. This is an example intended to show ways to improve the pipeline for historical texts.


In [15]:
#Import modules used in this notebook

import spacy
from spacy.lang.es import Spanish 
from spacy import displacy

import json
from copy import deepcopy

## Understanding the Pipeline 

<div>
<img src=https://spacy.io/pipeline-7a14d4edd18f3edfee8f34393bff2992.svg  width="700"/>
</div>
When analyzing a text, Spacy takes a string and passes it through a chain of processes. First, it tokenizes the text, separating it into the individual words contained in the document. Then, it applies other processes (by default, the tagger, the parser, and the named entity recognizer) to your document. These components are independent and can be disabled or dropped, depending on your NLP needs. New, custom components can be added (and the tokenizer can also be modified).

We are interested in improving the performance of the NER component, which translates each token into a numerical vector calculated from each token's prefix, suffix, norm and shape attributes. Early Modern languages display a lack of standardization of spellings. Thus, several strings might in reality represent the same word (in our documents, for example, Sevilla might be spelled Seuilla, seuilla, Sebilla, sebilla, Sevilla or sevilla). Standardizing spelling would help the model recognize the tokens as equivalent, improving predictions. 

To improve performance in this regard, we add a new component to the pipeline, after tokenization but before NER is applied.

In this notebook, we add a custom component that takes historical texts with multiple variations in spelling, and normalizes spelling of the norm attribute to bring it closer to modern usage. Because the Spacy model used was trained on texts written in modern Spanish, this should improve performance. 


In [16]:
#First, load the model
nlp= spacy.load('es_core_news_md')

In [17]:
#Create model copy to modify (allows us to compare results before and after modification)
nlp1= deepcopy(nlp)

## Modifying the Pipeline

The first step was to create a dictionary of early modern Spanish words. We did this by tokenizing a subset of our documents and making a list of unique tokens. In openrefine, we then duplicated the tokens and edited spellings to coincide with their modern form. We then structured the document to be in the shape of a dictionary ('key':'value' pairs), with the original token as a key and the new spelling as a value. (Full process available [here]()).

These are available in a file saved separately, which we call here.

In [26]:
# read our normalization dictionary from JSON file
with open('normalizeddict.json', 'r', encoding='utf-8') as fp2:
    normalizeddict = json.load(fp2)

Next, we define the function that updates the .norm attribute for each token:

In [19]:
#Define and add pipeline component that updates .norm attribute

def add_custom_norms(doc):
    for token in doc:
        if token.text in normalizeddict:
            token.norm_ = normalizeddict[token.text]
    return doc

And add the component to the pipeline, after tokenizing but before any other process:

In [20]:
#Add component to the pipeline

nlp1.add_pipe(add_custom_norms, first=True)

That is it! This is one of many modifications that can be done to the pipeline. Whenever you call the pipeline within the session, you are calling the new modified version. If you want to keep working with the modified pipeline in other sessions, you will want to save the updated model (instructions [here](https://github.com/FelipeAdeT/ArtMarketsofSeville/blob/master/Notebooks/Training%20Spacy's%20NER.ipynb), under "Saving the Updated Model").

At some point, you also might want to remove, rename replace or disable the component:

In [ ]:
#To remove or replace the component:
nlp.remove_pipe("add_custom_norms")
nlp.rename_pipe("add_custom_norms", "spelling_normalization")
nlp.replace_pipe("add_custom_norms", new_component) #With a new component defined as above

#To disable the pipeline while you are processing a batch of documents:
with nlp.disable_pipes("add_custom_norms"): #You can include any list of components, separated by commas
    doc = nlp("I won't be normalized")

## Viewing pipeline component results

The next steps are intented to help visualize the changes made above. First, we look at one document with its original spelling and its normalized spelling. Then, we look at how the model's performance changes with the inclusion of the new pipeline component.

In [28]:
#Processing a text with the unmodified and the modified pipelines.

doc=nlp("juan baptista bazquez escultor vzo en san pedro como principal e diego de çamora pintor de ymagineria como su fiador otorgamos que somos concertados con juan rruiz de porras vzo de la ciudad de Ronda en nonbre de los texedores de paños e lienços de Ronda en tal manera que nos obligamos de hazer de talla e pintura un san marcos que tenga sin la peana bara e media de alto e la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de yr la ynsinia que es un toro del tamaño que le conuenga. el qual santo a de yr vestido como de apostol y a de ser de madera de pino de sigura a de yr hueco para que se pueda lleuar en prosesion y todo el y la peana a de yr dorado y estofado todos los uestidos conforme e de la misma manera questa eftofado una ymagen de señor san josehpe questa en la casa profesa de la conpañia de jesus en la capilla mayor en un altar y nos obligamos a hazer una caxa de madera de pino en que vaya metido para lleuar de aqui a Ronda y mas nos obligamos a hazer unas pariguelas para sacallo en prosesion las tablas de pino y los braços de borne jaspeadas de verde a el olio y el santo a de llevar en la mano ysquierda un libro dorado y en la mano derecha una pluma y en la cabeça una diadema dorada todo ello de buena obra y nos obligamos de lo dar acabado de aqui a el dia de san juan bautista del mes de junio deste año por precio todo ello de quarenta ducados y rrecibo para en quenta 17 ducados.")
doc1=nlp1("juan baptista bazquez escultor vzo en san pedro como principal e diego de çamora pintor de ymagineria como su fiador otorgamos que somos concertados con juan rruiz de porras vzo de la ciudad de Ronda en nonbre de los texedores de paños e lienços de Ronda en tal manera que nos obligamos de hazer de talla e pintura un san marcos que tenga sin la peana bara e media de alto e la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de yr la ynsinia que es un toro del tamaño que le conuenga. el qual santo a de yr vestido como de apostol y a de ser de madera de pino de sigura a de yr hueco para que se pueda lleuar en prosesion y todo el y la peana a de yr dorado y estofado todos los uestidos conforme e de la misma manera questa eftofado una ymagen de señor san josehpe questa en la casa profesa de la conpañia de jesus en la capilla mayor en un altar y nos obligamos a hazer una caxa de madera de pino en que vaya metido para lleuar de aqui a Ronda y mas nos obligamos a hazer unas pariguelas para sacallo en prosesion las tablas de pino y los braços de borne jaspeadas de verde a el olio y el santo a de llevar en la mano ysquierda un libro dorado y en la mano derecha una pluma y en la cabeça una diadema dorada todo ello de buena obra y nos obligamos de lo dar acabado de aqui a el dia de san juan bautista del mes de junio deste año por precio todo ello de quarenta ducados y rrecibo para en quenta 17 ducados.")

In [22]:
#Displaying unmodified text
for token in doc1:
    print(token.text,end=' ')

juan baptista bazquez escultor vzo en san pedro como principal e diego de çamora pintor de ymagineria como su fiador otorgamos que somos concertados con juan rruiz de porras vzo de la ciudad de Ronda en nonbre de los texedores de paños e lienços de Ronda en tal manera que nos obligamos de hazer de talla e pintura un san marcos que tenga sin la peana bara e media de alto e la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de yr la ynsinia que es un toro del tamaño que le conuenga . el qual santo a de yr vestido como de apostol y a de ser de madera de pino de sigura a de yr hueco para que se pueda lleuar en prosesion y todo el y la peana a de yr dorado y estofado todos los uestidos conforme e de la misma manera questa eftofado una ymagen de señor san josehpe questa en la casa profesa de la conpañia de jesus en la capilla mayor en un altar y nos obligamos a hazer una caxa de madera de pino en que vaya metido para lleuar de aqui a Ronda y mas nos obligamos a haze

In [29]:
#Displaying normalized text
for token in doc1:
    print(token.norm_, end=' ')

juan bautista bazquez escultor vecino en san pedro como principal y diego de zamora pintor de imagineria como su fiador otorgamos que somos concertados con juan rruiz de porras vecino de la ciudad de ronda en nombre de los texedores de paños y lienzos de ronda en tal manera que nos obligamos de hacer de talla y pintura un san marcos que tenga sin la peana vara y media de alto y la peana de medio palmo con sus molduras y al pie del santo sobre la peana a de ir la insignia que es un toro del tamaño que le convenga . el cual santo a de ir vestido como de apostol y a de ser de madera de pino de sigura a de ir hueco para que se pueda llevar en prosesion y todo el y la peana a de ir dorado y estofado todos los uestidos conforme y de la misma manera que esta eftofado una imagen de señor san josehpe que esta en la casa profesa de la compañia de jesus en la capilla mayor en un altar y nos obligamos a hacer una caja de madera de pino en que vaya metido para llevar de aqui a ronda y mas nos oblig

In [24]:
#Displaying recognized entities in this document, with unmodified pipeline
displacy.render(doc, style='ent', jupyter= True)

In [30]:
#Displaying recognized entities in the document, with modified pipeline
displacy.render(doc1, style='ent', jupyter= True)

As can be seen, there are some differences in the way the NER pipeline component identifies entities in the document, though whether there it is unclear whether this is an improvement. 

This combined with training should result in improved performance.